In [3]:
# imports
import pandas as pd
from sqlalchemy import create_engine

In [1]:
# start Abbys section

Bringing in Target Location file

In [2]:
target_df = pd.read_csv('target.csv', encoding='utf-8')
target_df.head()

,X.locale,Address.AddressLine1,Address.AddressLine2,Address.City,Address.CountryName,Address.County,Address.FormattedAddress,Address.IntersectionDescription,Address.Latitude,Address.Longitude,...,Is24Hours.Sat,IsOpen.Sat,Summary.Sat,ThruTime.Sat,BeginTime.Sun,Is24Hours.Sun,IsOpen.Sun,Summary.Sun,ThruTime.Sun,AllCapability
0,en-US,400 Oxford Exchange Blvd,NaN,Oxford,United States,Calhoun,"400 Oxford Exchange Blvd, Oxford, AL 36203-3459",SEC I-20 & Golden Creek Rd,33.608825,-85.783182,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'CafÈ', 'Mobile Kiosk', 'Star..."
1,en-US,2640 Enterprise Dr,NaN,Opelika,United States,Lee,"2640 Enterprise Dr, Opelika, AL 36801-1511",NWC I-85 & Hwy 280,32.618108,-85.410636,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'Cafe-Pizza', 'Fresh Grocery'..."
2,en-US,1129 S Park St,NaN,Carrollton,United States,Carroll,"1129 S Park St, Carrollton, GA 30117-4429",NWC US 27 & Hwy 166 Bypass,33.558920,-85.076473,...,False,True,8:00 a.m.-10:00 p.m.,22:00:00,8:00:00,False,True,8:00 a.m.-9:00 p.m.,21:00:00,NaN
3,en-US,555 Bullsboro Dr,NaN,Newnan,United States,Coweta,"555 Bullsboro Dr, Newnan, GA 30265-1045",SWC I-85 & Rte 34,33.392907,-84.758875,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'CafÈ', 'Fresh Grocery', 'Mob..."
4,en-US,1591 Bradley Park Dr,NaN,Columbus,United States,Muscogee,"1591 Bradley Park Dr, Columbus, GA 31904-3071",NEC Bradley Pk & JR Allen Pkwy,32.534649,-84.971073,...,False,True,8:00 a.m.-11:00 p.m.,23:00:00,8:00:00,False,True,8:00 a.m.-10:00 p.m.,22:00:00,"['CVS pharmacy', 'CafÈ', 'Mobile Kiosk', 'Phot..."


In [3]:
removecolumns_target_df = target_df[['ID', 'Address.AddressLine1', 'Address.City', 'Address.PostalCode', 'Address.Subdivision']]
removecolumns_target_df.head()

,ID,Address.AddressLine1,Address.City,Address.PostalCode,Address.Subdivision
0,2153,400 Oxford Exchange Blvd,Oxford,36203-3459,AL
1,1499,2640 Enterprise Dr,Opelika,36801-1511,AL
2,1073,1129 S Park St,Carrollton,30117-4429,GA
3,1223,555 Bullsboro Dr,Newnan,30265-1045,GA
4,1179,1591 Bradley Park Dr,Columbus,31904-3071,GA


In [4]:
renamed_target_df = removecolumns_target_df.rename(columns={
    'ID': 'target_store_id',
    'Address.AddressLine1': 'street_address',
    'Address.City': 'city',
    'Address.PostalCode': 'zip_code',
    'Address.Subdivision': 'state_abr'
})
renamed_target_df.head()

,target_store_id,street_address,city,zip_code,state_abr
0,2153,400 Oxford Exchange Blvd,Oxford,36203-3459,AL
1,1499,2640 Enterprise Dr,Opelika,36801-1511,AL
2,1073,1129 S Park St,Carrollton,30117-4429,GA
3,1223,555 Bullsboro Dr,Newnan,30265-1045,GA
4,1179,1591 Bradley Park Dr,Columbus,31904-3071,GA


In [5]:
renamed_target_df['striped_zip'] = renamed_target_df['zip_code'].str.split('-', n = 1, expand = True)[0]

newzip_target_df = renamed_target_df.drop(columns=('zip_code'))
newzip_target_df

,target_store_id,street_address,city,state_abr,striped_zip
0,2153,400 Oxford Exchange Blvd,Oxford,AL,36203
1,1499,2640 Enterprise Dr,Opelika,AL,36801
2,1073,1129 S Park St,Carrollton,GA,30117
3,1223,555 Bullsboro Dr,Newnan,GA,30265
4,1179,1591 Bradley Park Dr,Columbus,GA,31904
...,...,...,...,...,...
1824,2239,22832 US Highway 281 N,San Antonio,TX,78258
1825,2429,135 Creekside Way,New Braunfels,TX,78130
1826,2438,700 Barnes Dr,San Marcos,TX,78666
1827,2725,5188 Kyle Center Dr,Kyle,TX,78640


In [33]:
rds_connection_string = "postgres:postgres@localhost:5432/store_locations_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [34]:
engine.table_names()

['target_locations', 'starbucks_menu']

In [11]:
newzip_target_df.to_sql(name="target_locations", con=engine, if_exists='append', index=False)

In [12]:
pd.read_sql_query('select * from target_locations', con=engine).head()

,target_store_id,street_address,city,state_abr,striped_zip
0,2153,400 Oxford Exchange Blvd,Oxford,AL,36203
1,1499,2640 Enterprise Dr,Opelika,AL,36801
2,1073,1129 S Park St,Carrollton,GA,30117
3,1223,555 Bullsboro Dr,Newnan,GA,30265
4,1179,1591 Bradley Park Dr,Columbus,GA,31904


Bringin in Starbucks Menu file

In [5]:
starbucks_menu = pd.read_csv('starbucks_drinkMenu_expanded.csv', encoding='utf-8')
starbucks_menu.dtypes

Beverage_category             object
Beverage                      object
Beverage_prep                 object
Calories                       int64
 Total Fat (g)                object
Trans Fat (g)                float64
Saturated Fat (g)            float64
 Sodium (mg)                   int64
 Total Carbohydrates (g)       int64
Cholesterol (mg)               int64
 Dietary Fibre (g)             int64
 Sugars (g)                    int64
 Protein (g)                 float64
Vitamin A (% DV)              object
Vitamin C (% DV)              object
 Calcium (% DV)               object
Iron (% DV)                   object
Caffeine (mg)                 object
dtype: object

In [7]:
starbucks_menu.head()

,Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
0,Coffee,Brewed Coffee,Short,3,0.1,0.0,0.0,0,5,0,0,0,0.3,0%,0%,0%,0%,175
1,Coffee,Brewed Coffee,Tall,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
2,Coffee,Brewed Coffee,Grande,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
3,Coffee,Brewed Coffee,Venti,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
4,Classic Espresso Drinks,Caff√® Latte,Short Nonfat Milk,70,0.1,0.1,0.0,5,75,10,0,9,6.0,10%,0%,20%,0%,75


In [18]:
# menu_cleaned_df = starbucks_menu.rename(columns={
#     ' Total Fat (g)':'Total_Fat_g',
#     'Trans Fat (g)':'Trans_Fat_g',
#     'Saturated Fat (g)':'Saturated_Fat_g',
#     'Sodium (mg)':'Sodium_mg',
#     'Total Carbohydrates (g)':'Total_Carbohydrates_g',
#     'Cholesterol (mg)':'Cholesterol_mg',
#     'Dietary Fibre (g)':'Dietary_Fibre_g',
#     'Sugars (g)':'Sugars_g',
#     'Protein (g)':'Protein_g',
#     'Vitamin A (% DV)':'Vitamin_A_DV',
#     'Vitamin C (% DV)':'Vitamin_C_DV',
#     'Calcium (% DV)':'Calcium_DV',
#     'Iron (% DV)':'Iron_DV',
#     'Caffeine (mg)':'Caffeine_mg'
# })
# menu_cleaned_df.head()

In [ ]:
# create table starbucks_menu(
# 	Beverage_category text,
# 	Beverage text,
# 	Beverage_prep text,
# 	Calories int,
# 	Total_Fat_g text,
# 	Trans_Fat_g decimal,
# 	Saturated_Fat_g decimal,
# 	Sodium_mg int,
# 	Total_Carbohydrates_g int,
# 	Cholesterol_mg int,
# 	Dietary_Fibre_g int,
# 	Sugars_g int,
# 	Protein_g decimal,
# 	Vitamin_A_DV text,
# 	Vitamin_C_DV text,
# 	Calcium_DV text,
# 	Iron_DV text,
# 	Caffeine_mg text
# );

In [20]:
original_column_titles = list(starbucks_menu.columns.values)
original_column_titles

['Beverage_category',
 'Beverage',
 'Beverage_prep',
 'Calories',
 ' Total Fat (g)',
 'Trans Fat (g) ',
 'Saturated Fat (g)',
 ' Sodium (mg)',
 ' Total Carbohydrates (g) ',
 'Cholesterol (mg)',
 ' Dietary Fibre (g)',
 ' Sugars (g)',
 ' Protein (g) ',
 'Vitamin A (% DV) ',
 'Vitamin C (% DV)',
 ' Calcium (% DV) ',
 'Iron (% DV) ',
 'Caffeine (mg)']

In [53]:
clean_column_titles = []

for column in original_column_titles:
    columnLower = column.lower()
    columnSpace = columnLower.strip()
    columnPercent = columnSpace.replace('%',"")
    columnOpen = columnPercent.replace("(", "")
    columnClose = columnOpen.replace(")", "")
#     columnDoubleSpace = columnClose.replace("  ", "")
    columnUnderscore = columnClose.replace(" ", "_")
    columnDouble = columnUnderscore.replace("__", "_")
    
    clean_column_titles.append(columnDouble)
    
clean_column_titles

['beverage_category',
 'beverage',
 'beverage_prep',
 'calories',
 'total_fat_g',
 'trans_fat_g',
 'saturated_fat_g',
 'sodium_mg',
 'total_carbohydrates_g',
 'cholesterol_mg',
 'dietary_fibre_g',
 'sugars_g',
 'protein_g',
 'vitamin_a_dv',
 'vitamin_c_dv',
 'calcium_dv',
 'iron_dv',
 'caffeine_mg']

In [54]:
# dict(zip(a, b))
column_dict = dict(zip(original_column_titles,clean_column_titles))
column_dict

{'Beverage_category': 'beverage_category',
 'Beverage': 'beverage',
 'Beverage_prep': 'beverage_prep',
 'Calories': 'calories',
 ' Total Fat (g)': 'total_fat_g',
 'Trans Fat (g) ': 'trans_fat_g',
 'Saturated Fat (g)': 'saturated_fat_g',
 ' Sodium (mg)': 'sodium_mg',
 ' Total Carbohydrates (g) ': 'total_carbohydrates_g',
 'Cholesterol (mg)': 'cholesterol_mg',
 ' Dietary Fibre (g)': 'dietary_fibre_g',
 ' Sugars (g)': 'sugars_g',
 ' Protein (g) ': 'protein_g',
 'Vitamin A (% DV) ': 'vitamin_a_dv',
 'Vitamin C (% DV)': 'vitamin_c_dv',
 ' Calcium (% DV) ': 'calcium_dv',
 'Iron (% DV) ': 'iron_dv',
 'Caffeine (mg)': 'caffeine_mg'}

In [55]:
menu_cleaned_df = starbucks_menu.rename(columns=column_dict)
menu_cleaned_df.head()

,beverage_category,beverage,beverage_prep,calories,total_fat_g,trans_fat_g,saturated_fat_g,sodium_mg,total_carbohydrates_g,cholesterol_mg,dietary_fibre_g,sugars_g,protein_g,vitamin_a_dv,vitamin_c_dv,calcium_dv,iron_dv,caffeine_mg
0,Coffee,Brewed Coffee,Short,3,0.1,0.0,0.0,0,5,0,0,0,0.3,0%,0%,0%,0%,175
1,Coffee,Brewed Coffee,Tall,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
2,Coffee,Brewed Coffee,Grande,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
3,Coffee,Brewed Coffee,Venti,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
4,Classic Espresso Drinks,Caff√® Latte,Short Nonfat Milk,70,0.1,0.1,0.0,5,75,10,0,9,6.0,10%,0%,20%,0%,75


In [56]:
menu_cleaned_df.to_sql(name="starbucks_menu", con=engine, if_exists='append', index=False)

In [3]:
# end Abbys section

In [4]:
# start Umar section

In [ ]:
# end Umar section